In [39]:
import warnings
warnings.filterwarnings("ignore") 
import pandas as pd
fraud_dataset = pd.read_csv("Fraud_check.csv")
fraud_dataset.head(5)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [2]:
df = fraud_dataset.copy()

In [3]:
### Split my target variable
Ind_Var = df.drop("Taxable.Income", axis = 1 , inplace=False)
Target_Var = df.iloc[:,1]

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.3+ KB


In [7]:
for col in df.columns:
    if df[col].dtype == 'object':
        print("ColName  ", col," DataType====",df[col].dtype,df[col].unique())

ColName   Undergrad  DataType==== object ['NO' 'YES']
ColName   Marital.Status  DataType==== object ['Single' 'Divorced' 'Married']
ColName   Urban  DataType==== object ['YES' 'NO']


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in df.columns:
    if df[col].dtype == 'object':
        # Apply label encoding to the column
        df[col] = le.fit_transform(df[col])

In [10]:
df.head(3)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,0,2,68833,50047,10,1
1,1,0,33700,134075,18,1
2,0,1,36925,160205,30,1


In [12]:
x = df.drop("Taxable.Income", axis = 1 , inplace=False)
y = df.iloc[:,2]

In [19]:
y = y.apply(lambda x: 0 if x <= 30000 else 1)

In [24]:
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
x = SS.fit_transform(x)

In [25]:
x

array([[-1.040833  ,  1.16079994, -1.17852072, -0.62914254,  0.99335541],
       [ 0.96076892, -1.27444469,  0.50850002,  0.27636996,  0.99335541],
       [-1.040833  , -0.05682237,  1.03310911,  1.63463872,  0.99335541],
       ...,
       [-1.040833  , -1.27444469,  0.90969648, -1.76103318,  0.99335541],
       [ 0.96076892, -0.05682237,  1.43219749,  0.1631809 , -1.00668904],
       [-1.040833  , -1.27444469,  0.9915901 ,  0.04999184, -1.00668904]])

In [26]:
x = pd.DataFrame(x)
x.columns = list(Ind_Var)
x.head()

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban
0,-1.040833,1.160800,-1.178521,-0.629143,0.993355
1,0.960769,-1.274445,0.508500,0.276370,0.993355
2,-1.040833,-0.056822,1.033109,1.634639,0.993355
3,0.960769,1.160800,1.696831,-0.063197,0.993355
4,-1.040833,-0.056822,-1.630532,1.408261,-1.006689


In [27]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: Taxable.Income, dtype: int64

In [28]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((420, 5), (420,), (180, 5), (180,))

#

In [30]:
### Decision Tree Classifier
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix

# Fit the model using Decision Tree for training samples
model = DecisionTreeClassifier(criterion='entropy')
model.fit(x_train, y_train)

# Calculate the training and testing Parameters
train_acc = accuracy_score(y_train, model.predict(x_train))
test_acc  = accuracy_score(y_test, model.predict(x_test))

train_prec = precision_score(y_train, model.predict(x_train))
test_prec = precision_score(y_test, model.predict(x_test))

train_recall = recall_score(y_train, model.predict(x_train))
test_recall = recall_score(y_test, model.predict(x_test))

train_f1 = f1_score(y_train, model.predict(x_train))
test_f1 = f1_score(y_test, model.predict(x_test))

train_confusion = confusion_matrix(y_train, model.predict(x_train))
test_confusion = confusion_matrix(y_test, model.predict(x_test))

# Print the training and testing error
print("Training Accuracy:", (train_acc))
print("Testing Accuracy:", (test_acc))
print("")
print("Training Precision:", (train_prec))
print("Testing Precision:", (test_prec))
print("")
print("Training Recall:", (train_recall))
print("Testing Recall:", (test_recall))
print("")
print("Training F1:", (train_f1))
print("Testing F1:", (test_f1))
print("")
print("Training Confusion:", (train_confusion))
print("")
print("Testing Confusion:", (test_confusion))
print("")

Training Accuracy: 1.0
Testing Accuracy: 0.6222222222222222

Training Precision: 1.0
Testing Precision: 0.7906976744186046

Training Recall: 1.0
Testing Recall: 0.7132867132867133

Training F1: 1.0
Testing F1: 0.7499999999999999

Training Confusion: [[ 87   0]
 [  0 333]]

Testing Confusion: [[ 10  27]
 [ 41 102]]



In [31]:
#Get the number of nodes in the tree
num_nodes = model.tree_.node_count
# Get the depth of the tree
max_depth = model.tree_.max_depth

# Print the number of nodes and depth of the tree
print("Number of nodes:", num_nodes)
print("Depth of the tree:", max_depth)

Number of nodes: 241
Depth of the tree: 21


In [37]:
### Decision Tree Classifier
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix

# Fit the model using Decision Tree for training samples
model = DecisionTreeClassifier(criterion='entropy',max_depth=10)
model.fit(x_train, y_train)

# Calculate the training and testing Parameters
train_acc = accuracy_score(y_train, model.predict(x_train))
test_acc  = accuracy_score(y_test, model.predict(x_test))

train_prec = precision_score(y_train, model.predict(x_train))
test_prec = precision_score(y_test, model.predict(x_test))

train_recall = recall_score(y_train, model.predict(x_train))
test_recall = recall_score(y_test, model.predict(x_test))

train_f1 = f1_score(y_train, model.predict(x_train))
test_f1 = f1_score(y_test, model.predict(x_test))

train_confusion = confusion_matrix(y_train, model.predict(x_train))
test_confusion = confusion_matrix(y_test, model.predict(x_test))

# Print the training and testing error
print("Training Accuracy:", (train_acc))
print("Testing Accuracy:", (test_acc))
print("")
print("Training Precision:", (train_prec))
print("Testing Precision:", (test_prec))
print("")
print("Training Recall:", (train_recall))
print("Testing Recall:", (test_recall))
print("")
print("Training F1:", (train_f1))
print("Testing F1:", (test_f1))
print("")
print("Training Confusion:", (train_confusion))
print("")
print("Testing Confusion:", (test_confusion))
print("")

Training Accuracy: 0.8833333333333333
Testing Accuracy: 0.7222222222222222

Training Precision: 0.8756613756613757
Testing Precision: 0.7888198757763976

Training Recall: 0.993993993993994
Testing Recall: 0.8881118881118881

Training F1: 0.9310829817158931
Testing F1: 0.8355263157894737

Training Confusion: [[ 40  47]
 [  2 331]]

Testing Confusion: [[  3  34]
 [ 16 127]]



In [48]:
from sklearn.model_selection import GridSearchCV
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
model.predict(x_train)
model.predict(x_test)
# Define parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [8,10,11,12,13,14,16,18,20],
    'min_samples_split': [2, 5,6,7,8,9,10,15,20,25,30,40,50,55,60],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [2,4,6,8,10,12,14,16,18,10,22,24,26]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV
grid_search.fit(x, y)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

Best parameters found:  {'criterion': 'entropy', 'max_depth': 11, 'max_features': 'auto', 'min_samples_leaf': 18, 'min_samples_split': 40}


In [43]:
model = DecisionTreeClassifier(criterion='gini',max_depth=8,max_features='auto',min_samples_leaf=8,min_samples_split = 60)
model.fit(x_train, y_train)

# Calculate the training and testing Parameters
train_acc = accuracy_score(y_train, model.predict(x_train))
test_acc  = accuracy_score(y_test, model.predict(x_test))

train_prec = precision_score(y_train, model.predict(x_train))
test_prec = precision_score(y_test, model.predict(x_test))

train_recall = recall_score(y_train, model.predict(x_train))
test_recall = recall_score(y_test, model.predict(x_test))

train_f1 = f1_score(y_train, model.predict(x_train))
test_f1 = f1_score(y_test, model.predict(x_test))

train_confusion = confusion_matrix(y_train, model.predict(x_train))
test_confusion = confusion_matrix(y_test, model.predict(x_test))

# Print the training and testing error
print("Training Accuracy:", (train_acc))
print("Testing Accuracy:", (test_acc))
print("")
print("Training Precision:", (train_prec))
print("Testing Precision:", (test_prec))
print("")
print("Training Recall:", (train_recall))
print("Testing Recall:", (test_recall))
print("")
print("Training F1:", (train_f1))
print("Testing F1:", (test_f1))
print("")
print("Training Confusion:", (train_confusion))
print("")
print("Testing Confusion:", (test_confusion))
print("")

Training Accuracy: 0.7928571428571428
Testing Accuracy: 0.7611111111111111

Training Precision: 0.8106060606060606
Testing Precision: 0.7873563218390804

Training Recall: 0.963963963963964
Testing Recall: 0.958041958041958

Training F1: 0.8806584362139919
Testing F1: 0.864353312302839

Training Confusion: [[ 12  75]
 [ 12 321]]

Testing Confusion: [[  0  37]
 [  6 137]]



In [49]:
model = DecisionTreeClassifier(criterion='entropy',max_depth=11,max_features='auto',min_samples_leaf=18,min_samples_split = 40)
model.fit(x_train, y_train)

# Calculate the training and testing Parameters
train_acc = accuracy_score(y_train, model.predict(x_train))
test_acc  = accuracy_score(y_test, model.predict(x_test))

train_prec = precision_score(y_train, model.predict(x_train))
test_prec = precision_score(y_test, model.predict(x_test))

train_recall = recall_score(y_train, model.predict(x_train))
test_recall = recall_score(y_test, model.predict(x_test))

train_f1 = f1_score(y_train, model.predict(x_train))
test_f1 = f1_score(y_test, model.predict(x_test))

train_confusion = confusion_matrix(y_train, model.predict(x_train))
test_confusion = confusion_matrix(y_test, model.predict(x_test))

# Print the training and testing error
print("Training Accuracy:", (train_acc))
print("Testing Accuracy:", (test_acc))
print("")
print("Training Precision:", (train_prec))
print("Testing Precision:", (test_prec))
print("")
print("Training Recall:", (train_recall))
print("Testing Recall:", (test_recall))
print("")
print("Training F1:", (train_f1))
print("Testing F1:", (test_f1))
print("")
print("Training Confusion:", (train_confusion))
print("")
print("Testing Confusion:", (test_confusion))
print("")

Training Accuracy: 0.7928571428571428
Testing Accuracy: 0.7944444444444444

Training Precision: 0.7928571428571428
Testing Precision: 0.7944444444444444

Training Recall: 1.0
Testing Recall: 1.0

Training F1: 0.8844621513944223
Testing F1: 0.8854489164086687

Training Confusion: [[  0  87]
 [  0 333]]

Testing Confusion: [[  0  37]
 [  0 143]]



In [51]:
train_acc = []
test_acc = []
train_prec = []
test_acc = []
train_prec = []
test_prec = []
train_recall = []
test_recall = []
train_f1 = []
test_f1 = []

for i in range(1,100,1):
    model = DecisionTreeClassifier(criterion='entropy',max_depth=11,max_features='auto',min_samples_leaf=18,min_samples_split = 40)
    model.fit(x_train, y_train)
    # Calculate the training and testing Parameters
    train_acc.append(accuracy_score(y_train, model.predict(x_train)))
    test_acc.append(accuracy_score(y_test, model.predict(x_test)))
    train_prec.append(precision_score(y_train, model.predict(x_train)))
    test_prec.append(precision_score(y_test, model.predict(x_test)))
    train_recall.append(recall_score(y_train, model.predict(x_train)))
    test_recall.append(recall_score(y_test, model.predict(x_test)))
    train_f1.append(f1_score(y_train, model.predict(x_train)))
    test_f1.append(f1_score(y_test, model.predict(x_test)))


# Print the training and testing error
print("Training Accuracy:", np.mean(train_acc))
print("Testing Accuracy:", np.mean(test_acc))
print("")
print("Training Precision:", np.mean(train_prec))
print("Testing Precision:", np.mean(test_prec))
print("")
print("Training Recall:", np.mean(train_recall))
print("Testing Recall:", np.mean(test_recall))
print("")
print("Training F1:", np.mean(train_f1))
print("Testing F1:", np.mean(test_f1))
print("")

Training Accuracy: 0.7930014430014429
Testing Accuracy: 0.7877665544332212

Training Precision: 0.7959238715460608
Testing Precision: 0.7944532069452533

Training Recall: 0.9939636606303273
Testing Recall: 0.9886275340820796

Training F1: 0.8839012281589873
Testing F1: 0.880831438226889



In [50]:
from sklearn.ensemble import BaggingClassifier
train_acc = []
test_acc = []
train_prec = []
test_acc = []
train_prec = []
test_prec = []
train_recall = []
test_recall = []
train_f1 = []
test_f1 = []

for i in range(1,400,1):
    model = DecisionTreeClassifier(criterion='gini',max_depth= 11,max_features='auto',min_samples_leaf=18,min_samples_split = 40)
    model.fit(x_train, y_train)
    bag_reg = BaggingClassifier(estimator=model, n_estimators=200, max_features=0.6,max_samples=0.7,random_state=i)
    bag_reg.fit(x_train, y_train)
    train_acc.append(accuracy_score(y_train, bag_reg.predict(x_train)))
    test_acc.append(accuracy_score(y_test, bag_reg.predict(x_test)))
    train_prec.append(precision_score(y_train, bag_reg.predict(x_train)))
    test_prec.append(precision_score(y_test, bag_reg.predict(x_test)))
    train_recall.append(recall_score(y_train, bag_reg.predict(x_train)))
    test_recall.append(recall_score(y_test, bag_reg.predict(x_test)))
    train_f1.append(f1_score(y_train, bag_reg.predict(x_train)))
    test_f1.append(f1_score(y_test, bag_reg.predict(x_test)))

# Print the training and testing error
print("Training Accuracy:", np.mean(train_acc))
print("Testing Accuracy:", np.mean(test_acc))
print("")
print("Training Precision:", np.mean(train_prec))
print("Testing Precision:", np.mean(test_prec))
print("")
print("Training Recall:", np.mean(train_recall))
print("Testing Recall:", np.mean(test_recall))
print("")
print("Training F1:", np.mean(train_f1))
print("Testing F1:", np.mean(test_f1))
print("")


Training Accuracy: 0.7928571428571429
Testing Accuracy: 0.7944444444444445

Training Precision: 0.7928571428571429
Testing Precision: 0.7944444444444445

Training Recall: 1.0
Testing Recall: 1.0

Training F1: 0.8844621513944221
Testing F1: 0.8854489164086683



NameError: name 'df' is not defined